In [ ]:
"""This module defines all the functions for tax analysis"""


def calculate_tax(income : float, regime : str) -> float :
    """
    Calculates income tax using different regimes for individuals below 60 years of age.

    Inputs:
         income : CTC (cost to company) - Employer's PF - gratuity
         regime : "old"   - traditional scheme
                  "new"   - tax regime introduced in budget 2020
                  "newer" - tax regime introduced in budget 2023
    Output:
        income tax 
    """
    standard_deduction = 50000 # This is provided in all the regimes irrespective of any exemption claimed
    income = income - standard_deduction

    if regime == "old":
        slabs = [
        (250000, 0, 0.0),   # No tax up to Rs. 2.5 lakh
        (500000, 250000, 0.05),  # 5% tax on income between Rs. 2.5 lakh to Rs. 5 lakh
        (1000000, 500000, 0.2),  # 20% tax on income between Rs. 5 lakh to Rs. 10 lakh
        (float('inf'), 1000000, 0.3)  # 30% tax on income above Rs. 10 lakh
        ]
    elif regime == "new":
        slabs = [
        (250000, 0, 0.0),   # No tax up to Rs. 2.5 lakh
        (500000, 250000, 0.05),  # 5% tax on income between Rs. 2.5 lakh to Rs. 5 lakh
        (750000, 500000, 0.1),  # 10% tax on income between Rs. 5 lakh to Rs. 7.5 lakh
        (1000000, 750000, 0.15),  # 15% tax on income between Rs. 7.5 lakh to Rs. 10 lakh
        (1250000, 1000000, 0.2),  # 20% tax on income between Rs. 10 lakh to Rs. 12.5 lakh
        (1500000, 1250000, 0.25),  # 25% tax on income between Rs. 12.5 lakh to Rs. 15 lakh
        (float('inf'), 1500000, 0.3)  # 30% tax on income above Rs. 15 lakh
        ]
    elif regime == "newer":
        slabs = [
        (300000, 0, 0.0),   # No tax up to Rs. 2.5 lakh
        (600000, 300000, 0.05),  # 5% tax on income between Rs. 3 lakh to Rs. 6 lakh
        (900000, 600000, 0.1),  # 10% tax on income between Rs. 6 lakh to Rs. 9 lakh
        (1200000, 900000, 0.15),  # 15% tax on income between Rs. 9 lakh to Rs. 12 lakh
        (1500000, 1200000, 0.2),  # 20% tax on income between Rs. 12 lakh to Rs. 15 lakh
        (float('inf'), 1500000, 0.3)  # 30% tax on income above Rs. 15 lakh
        ]
        if income <= 700000:
            return 0

    slab_tax = []

    # Calculate tax
    tax = 0.0
    for i, (slab_ceiling, slab_floor, slab_rate) in enumerate(slabs):
        if income <= slab_ceiling:
            tax += (income - slab_floor) * slab_rate
            slab_tax.append((income - slab_floor) * slab_rate)
            break
        else:
            tax += (slab_ceiling - slab_floor) * slab_rate
            slab_tax.append((slab_ceiling - slab_floor) * slab_rate)

    tax = tax * (1.04) # Adding health and education cess
            
    #print(f"Total income tax with income Rs. {income:.2f} is Rs. {tax:.2f} \n Tax summary {slab_tax}")
    return tax

def calculate_income_old_regime(tax : float) -> float:
    """
    This function calculates the inverse of calculate_income_tax(income, "old") function
    It takes in income tax and returns income

    
    Mathematical technicality:
    
    tax = 0 is not included in the domain of this function because the calculate_income_tax(income, "old") function
    is not one-one. For example:
    
    calculate_income_tax(200000, "old") = 0
    calculate_income_tax(250000, "old") = 0

    Therefore, calculate_income_tax(income, "old") function is not invertible in the domain -> [0, 300000]
    """
    tax = tax/1.04
    if tax == 0:
        return "Anywhere between 0 and 300000"
    elif 0 < tax <= 12500:
        income = tax / 0.05 + 300000 # (300000 = 250000 + 50000 -> income ceil in first slab + standard deduction)
        return income 
    elif 12500 < tax <= 112500:
        income = (tax - 12500) / 0.2 + 550000 # (550000 = 500000 + 50000 -> income ceil in second slab + standard deduction)
        return income
    else:
        income = (tax - 112500) / 0.3 + 1050000 # (1050000 = 1000000 + 50000 -> income ceil in third slab + standard deduction)
        return income
    
def minimum_exemption_for_tax_regime_equivalence(income : float, regime : str) -> float:
    """ 
    minimum exemption required after excluding standard deduction is given by:
    N(.) -> function to calculate income tax using new regime 
    O(.) -> function to calculate income tax using old regime

    -> N(I) = O(I-E)
    
    let function Q(.) be inverse of O(.) Therefore,
    E = I - Q(N(I))
    """
    return income - calculate_income_old_regime(calculate_tax(income, regime))











